## 扒一扒疫苗类生物制药公司

In [1]:
# 获取用户名和token
import os
user  = os.environ.get('QUANTOS_USER')
token = os.environ.get('QUANTOS_TOKEN')

In [2]:
# 登陆系统
from jaqs.data import DataApi
api = DataApi('tcp://data.quantos.org:8910')
api.login(user, token)

('username: 18612562791', '0,')

In [3]:
# 获取申万一级行业=医药生物，二级行业=生物制品 的公司
df, msg = api.query(view='lb.secIndustry', filter='industry_src=SW&industry1_name=医药生物&industry2_name=生物制品')
df2 = df[(df.out_date == '')]

def get_market(x):
    if len(x) != 9:
        return ''
    
    array = x.split('.')
    if len(array)>1:
        return array[1]
    
    return ''

df2['market'] = df2['symbol'].apply(lambda x : get_market(x))
df3 = df2[(df2.market == 'SH') | (df2.market == 'SZ') ]

E:\QuantosFinanceTerminal\python\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [4]:
# 构建universe集合
universe = ",".join(df3['symbol'])
df, msg = api.query(view='jz.instrumentInfo', filter = 'symbol=%s' % universe)
df_universe = df.set_index('symbol')
df_universe

,list_date,name
symbol,,
002007.SZ,20040625,华兰生物
002038.SZ,20040909,双鹭药业
002252.SZ,20080623,上海莱士
000661.SZ,19961218,长春高新
002030.SZ,20040809,达安基因
000004.SZ,19910114,国农科技
000518.SZ,19930908,四环生物
002166.SZ,20070913,莱茵生物
002581.SZ,20110520,未名医药


In [5]:
#股票今年的收益分析

df, msg = api.daily(symbol=universe, start_date=20171229, end_date=20180719, adjust_mode='post')
df_start = df[df.trade_date == 20171229]
df_end   = df[df.trade_date == 20180719]
df_start.set_index('symbol', inplace=True)
df_end.set_index('symbol', inplace=True)
ts_return = df_end['close'] / df_start['close'] - 1

df_universe['ytd_return'] = ts_return
df_universe

,list_date,name,ytd_return
symbol,,,
002007.SZ,20040625,华兰生物,0.297802
002038.SZ,20040909,双鹭药业,0.149072
002252.SZ,20080623,上海莱士,-0.015767
000661.SZ,19961218,长春高新,0.322288
002030.SZ,20040809,达安基因,-0.044832
000004.SZ,19910114,国农科技,-0.207328
000518.SZ,19930908,四环生物,-0.403509
002166.SZ,20070913,莱茵生物,-0.267009
002581.SZ,20110520,未名医药,-0.491393


In [7]:
# 市值分析
import pandas as pd
df, msg = api.query(view='lb.secDailyIndicator',fields='total_mv,pe_ttm', filter='start_date=20180719&end_date=20180719&symbol=%s'%universe)
df.set_index('symbol', inplace=True)

df_universe = pd.concat([df_universe, df], axis=1)
df_universe

,list_date,name,ytd_return,pe_ttm,total_mv
000004.SZ,19910114,国农科技,-0.207328,126.1941,1.489746e+05
000403.SZ,19960628,ST生化,0.008073,173.7032,8.169151e+05
000518.SZ,19930908,四环生物,-0.403509,28312.5721,4.200589e+05
000661.SZ,19961218,长春高新,0.322288,56.2875,4.100216e+06
002007.SZ,20040625,华兰生物,0.297802,41.5926,3.214383e+06
002030.SZ,20040809,达安基因,-0.044832,147.2105,1.284632e+06
002038.SZ,20040909,双鹭药业,0.149072,40.9193,2.409478e+06
002166.SZ,20070913,莱茵生物,-0.267009,15.9789,3.375812e+05
002252.SZ,20080623,上海莱士,-0.015767,0.0000,9.710462e+06
002581.SZ,20110520,未名医药,-0.491393,16.3452,6.432422e+05


In [9]:
# 财务分析
api.set_timeout(60)
df, msg = api.query(view='lb.finIndicator',fields='grossprofitmargin,saleexpensetogr,adminexpensetogr,finaexpensetogr', filter='symbol=%s'%universe)
df = df[df.report_date == '20180331']
df.set_index('symbol', inplace=True)
df_universe['saleexpensetogr'] = df['saleexpensetogr']
df_universe['grossprofitmargin'] = df['grossprofitmargin']
#df_universe.to_csv('indicator.csv')
df_universe

,list_date,name,ytd_return,pe_ttm,total_mv,saleexpensetogr,grossprofitmargin
000004.SZ,19910114,国农科技,-0.207328,126.1941,1.489746e+05,72.8273,79.8477
000403.SZ,19960628,ST生化,0.008073,173.7032,8.169151e+05,11.5703,50.1735
000518.SZ,19930908,四环生物,-0.403509,28312.5721,4.200589e+05,45.6643,81.6867
000661.SZ,19961218,长春高新,0.322288,56.2875,4.100216e+06,41.2587,89.3683
002007.SZ,20040625,华兰生物,0.297802,41.5926,3.214383e+06,10.1724,61.3562
002030.SZ,20040809,达安基因,-0.044832,147.2105,1.284632e+06,15.1764,41.7778
002038.SZ,20040909,双鹭药业,0.149072,40.9193,2.409478e+06,30.9191,85.2441
002166.SZ,20070913,莱茵生物,-0.267009,15.9789,3.375812e+05,6.3085,37.5279
002252.SZ,20080623,上海莱士,-0.015767,0.0000,9.710462e+06,7.1704,67.6868
002581.SZ,20110520,未名医药,-0.491393,16.3452,6.432422e+05,46.3297,83.7061


In [11]:

# 002680.SZ 长生生物 销售费用分析

df, msg = api.query(view='lb.finIndicator',fields='grossprofitmargin,saleexpensetogr,adminexpensetogr,finaexpensetogr', filter='symbol=002680.SZ')
df.set_index('report_date', inplace=True)
df.sort_index(inplace=True)
#df['saleexpensetogr'].to_csv('002680.csv')
df['saleexpensetogr']

report_date
20091231     5.8737
20101231     5.0822
20111231     5.3183
20120331     4.4696
20120630     4.0139
20120930     4.1596
20121231     4.0651
20130331     4.8718
20130630     5.0181
20130930     5.3542
20131231     5.6118
20140331     4.8314
20140630     4.7897
20140930     8.3118
20141231     6.2873
20150331    10.5465
20150630    13.9137
20150930    11.0070
20151231    17.6736
20160331    16.9498
20160630    17.3820
20160930    24.9412
20161231    22.6800
20170331    28.2852
20170630    29.0441
20170930    34.9668
20171231    37.5287
20180331    33.8995
Name: saleexpensetogr, dtype: float64